In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import numpy as np

# Split data

In [3]:
import os
import shutil
import numpy as np


dataset_dir = r'Downloads\wheat final'
split_dir = r'Downloads\split wheat data' # Where you want to save the split data

# Create directories for split data
train_dir = os.path.join(split_dir, 'train')
val_dir = os.path.join(split_dir, 'val')
test_dir = os.path.join(split_dir, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Subdirectories for each class
classes = os.listdir(dataset_dir)
for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

# Helper function to split images
def split_data(src_dir, dest_dirs, split_ratios):
    # src_dir: source directory with images
    # dest_dirs: list of destination directories
    # split_ratios: list of ratios for each split
    
    all_files = []
    for cls in os.listdir(src_dir):
        class_dir = os.path.join(src_dir, cls)
        files = os.listdir(class_dir)
        all_files.extend([(os.path.join(class_dir, file), cls) for file in files])

    # Shuffle and split
    np.random.shuffle(all_files)
    total_files = len(all_files)
    train_split = int(total_files * split_ratios[0])
    val_split = int(total_files * (split_ratios[0] + split_ratios[1]))

    for file_path, cls in all_files[:train_split]:
        shutil.copy(file_path, os.path.join(dest_dirs[0], cls))
    for file_path, cls in all_files[train_split:val_split]:
        shutil.copy(file_path, os.path.join(dest_dirs[1], cls))
    for file_path, cls in all_files[val_split:]:
        shutil.copy(file_path, os.path.join(dest_dirs[2], cls))

# Split ratios: 70% training, 15% validation, 15% test
split_data(dataset_dir, [train_dir, val_dir, test_dir], [0.80, 0.10, 0.10])

# Set Up Data Generators

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to the split data
train_dir = r'Downloads\split wheat data\train'
val_dir = r'Downloads\split wheat data\val'
test_dir = r'Downloads\split wheat data\test'

# Create ImageDataGenerators for training, validation, and test datasets
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Adjust size as needed
    batch_size=32,
    class_mode='categorical'  # Use 'binary' if you have binary classification
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 5196 images belonging to 4 classes.
Found 1319 images belonging to 4 classes.
Found 1289 images belonging to 4 classes.


# Model Training And Train Model

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Define your model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')  # Number of classes
])

# Compile the model
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Adjust number of epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


C:\Users\kulka\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


C:\Users\kulka\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


162/162 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.5355 - loss: 1.7497 - val_accuracy: 0.7919 - val_loss: 0.5469
Epoch 2/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 886us/step - accuracy: 0.5625 - loss: 0.8235 - val_accuracy: 1.0000 - val_loss: 0.1418
Epoch 3/10


C:\Users\kulka\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


162/162 ━━━━━━━━━━━━━━━━━━━━ 194s 1s/step - accuracy: 0.8207 - loss: 0.4727 - val_accuracy: 0.8636 - val_loss: 0.3688
Epoch 4/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 476us/step - accuracy: 0.7812 - loss: 0.3780 - val_accuracy: 0.7143 - val_loss: 0.7592
Epoch 5/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - accuracy: 0.8825 - loss: 0.3127 - val_accuracy: 0.8666 - val_loss: 0.3303
Epoch 6/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 565us/step - accuracy: 0.8438 - loss: 0.2682 - val_accuracy: 0.7143 - val_loss: 0.9955
Epoch 7/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - accuracy: 0.9264 - loss: 0.1912 - val_accuracy: 0.9520 - val_loss: 0.1453
Epoch 8/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 557us/step - accuracy: 0.9062 - loss: 0.2928 - val_accuracy: 0.8571 - val_loss: 0.1222
Epoch 9/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 194s 1s/step - accuracy: 0.9698 - loss: 0.0906 - val_accuracy: 0.9428 - val_loss: 0.1766
Epoch 10/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 566us/step - accuracy: 0.9688 - loss: 0.2215 - val_accuracy: 

In [6]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Accuracy: {test_acc:.2f}')


40/40 ━━━━━━━━━━━━━━━━━━━━ 18s 440ms/step - accuracy: 0.9220 - loss: 0.2179
Test Accuracy: 0.93


In [11]:
model.save('Final_fixed.keras')